In [1]:
import imblearn
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://alikokoko71@gmail.com:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
query = '''
SELECT 
    f.film_id,
    f.title,
    fc.category_id,
    f.rental_rate,
    f.length,
    r.rental_id,
    r.rental_date
FROM
    film f
        JOIN
    film_category fc ON f.film_id = fc.film_id
        JOIN
    inventory i ON f.film_id = i.film_id
        JOIN
    rental r ON I.inventory_id = r.inventory_id;
    '''

df = pd.read_sql_query(query, engine)
df

,film_id,title,category_id,rental_rate,length,rental_id,rental_date
0,19,AMADEUS HOLY,1,0.99,113,10895,2005-08-02 01:16:59
1,19,AMADEUS HOLY,1,0.99,113,12268,2005-08-18 04:26:54
2,19,AMADEUS HOLY,1,0.99,113,3150,2005-06-20 20:35:28
3,19,AMADEUS HOLY,1,0.99,113,5072,2005-07-09 05:01:58
4,19,AMADEUS HOLY,1,0.99,113,9080,2005-07-30 08:02:39
...,...,...,...,...,...,...,...
16040,989,WORKING MICROCOSMOS,16,4.99,74,962,2005-05-30 18:45:17
16041,989,WORKING MICROCOSMOS,16,4.99,74,1292,2005-06-15 09:03:52
16042,989,WORKING MICROCOSMOS,16,4.99,74,4108,2005-07-07 06:38:31
16043,989,WORKING MICROCOSMOS,16,4.99,74,8871,2005-07-30 00:12:41


In [10]:
df.shape

(16045, 9)

In [14]:
df.dtypes

film_id                            int64
title                             object
category_id                        int64
rental_rate                      float64
length                             int64
rental_id                          int64
rental_date               datetime64[ns]
rented_last_month                   bool
rented_in_month_7_or_8              bool
dtype: object

In [12]:
df.describe()

,film_id,category_id,rental_rate,length,rental_id,rental_date
count,16045.000000,16045.000000,16045.000000,16045.000000,16045.000000,16045
mean,501.079277,8.364350,2.942509,114.969274,8025.871611,2005-07-23 18:15:35.130695040
min,1.000000,1.000000,0.990000,46.000000,1.000000,2005-05-24 22:53:30
25%,255.000000,4.000000,0.990000,81.000000,4014.000000,2005-07-07 00:58:54
50%,496.000000,8.000000,2.990000,114.000000,8026.000000,2005-07-28 16:05:38
75%,753.000000,13.000000,4.990000,148.000000,12038.000000,2005-08-17 21:24:47
max,1000.000000,16.000000,4.990000,185.000000,16050.000000,2023-12-11 19:51:10
std,288.528918,4.650768,1.649698,40.101750,4633.066013,NaN


In [13]:
df.isna().sum()

film_id                   0
title                     0
category_id               0
rental_rate               0
length                    0
rental_id                 0
rental_date               0
rented_last_month         0
rented_in_month_7_or_8    0
dtype: int64

In [16]:
df['category_id'].value_counts()

category_id
15    1179
2     1166
1     1112
14    1102
8     1096
7     1060
6     1050
9     1033
10     969
3      945
5      941
13     940
4      939
11     846
16     837
12     830
Name: count, dtype: int64

In [5]:
df['rented_last_month'] = (df['rental_date'].dt.month == (pd.to_datetime('today') - pd.DateOffset(months=1)).month)

In [6]:
df

,film_id,title,category_id,rental_rate,length,rental_id,rental_date,rented_last_month
0,19,AMADEUS HOLY,1,0.99,113,10895,2005-08-02 01:16:59,False
1,19,AMADEUS HOLY,1,0.99,113,12268,2005-08-18 04:26:54,False
2,19,AMADEUS HOLY,1,0.99,113,3150,2005-06-20 20:35:28,False
3,19,AMADEUS HOLY,1,0.99,113,5072,2005-07-09 05:01:58,False
4,19,AMADEUS HOLY,1,0.99,113,9080,2005-07-30 08:02:39,False
...,...,...,...,...,...,...,...,...
16040,989,WORKING MICROCOSMOS,16,4.99,74,962,2005-05-30 18:45:17,False
16041,989,WORKING MICROCOSMOS,16,4.99,74,1292,2005-06-15 09:03:52,False
16042,989,WORKING MICROCOSMOS,16,4.99,74,4108,2005-07-07 06:38:31,False
16043,989,WORKING MICROCOSMOS,16,4.99,74,8871,2005-07-30 00:12:41,False


In [7]:
df['rented_in_month_7_or_8'] = df['rental_date'].dt.month.isin([7, 8])

In [8]:
df

,film_id,title,category_id,rental_rate,length,rental_id,rental_date,rented_last_month,rented_in_month_7_or_8
0,19,AMADEUS HOLY,1,0.99,113,10895,2005-08-02 01:16:59,False,True
1,19,AMADEUS HOLY,1,0.99,113,12268,2005-08-18 04:26:54,False,True
2,19,AMADEUS HOLY,1,0.99,113,3150,2005-06-20 20:35:28,False,False
3,19,AMADEUS HOLY,1,0.99,113,5072,2005-07-09 05:01:58,False,True
4,19,AMADEUS HOLY,1,0.99,113,9080,2005-07-30 08:02:39,False,True
...,...,...,...,...,...,...,...,...,...
16040,989,WORKING MICROCOSMOS,16,4.99,74,962,2005-05-30 18:45:17,False,False
16041,989,WORKING MICROCOSMOS,16,4.99,74,1292,2005-06-15 09:03:52,False,False
16042,989,WORKING MICROCOSMOS,16,4.99,74,4108,2005-07-07 06:38:31,False,True
16043,989,WORKING MICROCOSMOS,16,4.99,74,8871,2005-07-30 00:12:41,False,True


In [17]:
df['rented_last_month'].value_counts()

rented_last_month
False    16044
True         1
Name: count, dtype: int64

In [19]:
df['rented_in_month_7_or_8'].value_counts()

rented_in_month_7_or_8
True     12395
False     3650
Name: count, dtype: int64

In [20]:
nan_counts = df.isna().sum()
print(nan_counts)

film_id                   0
title                     0
category_id               0
rental_rate               0
length                    0
rental_id                 0
rental_date               0
rented_last_month         0
rented_in_month_7_or_8    0
dtype: int64


In [21]:
X = df[['rental_rate', 'length', 'category_id']]
y = df['rented_in_month_7_or_8']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [22]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.7862262387036459
Confusion Matrix:
[[   0  686]
 [   0 2523]]
Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       686
        True       0.79      1.00      0.88      2523

    accuracy                           0.79      3209
   macro avg       0.39      0.50      0.44      3209
weighted avg       0.62      0.79      0.69      3209



C:\Users\Seijsener\.conda\New folderX\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Seijsener\.conda\New folderX\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Seijsener\.conda\New folderX\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


the logistic regression model is predicting only one class (True) for all instances, resulting in a high accuracy but poor performance on the False class.

In [23]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [24]:
X = df[['rental_rate', 'length', 'category_id']]
y = df['rented_in_month_7_or_8']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

LogisticRegression()

In [25]:
y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.4870676223122468
Confusion Matrix:
[[ 343  343]
 [1303 1220]]
Classification Report:
              precision    recall  f1-score   support

       False       0.21      0.50      0.29       686
        True       0.78      0.48      0.60      2523

    accuracy                           0.49      3209
   macro avg       0.49      0.49      0.45      3209
weighted avg       0.66      0.49      0.53      3209



The updated results show a more balanced performance across both classes, indicating improvement. 

However, there's still room for further optimization.

perform cross-validation using scikit-learn

In [26]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression()

cv_scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')

print(f'Cross-Validation Scores: {cv_scores}')
print(f'Mean Cross-Validation Accuracy: {cv_scores.mean()}')

Cross-Validation Scores: [0.49936693 0.51253482 0.51278805 0.50898962 0.50987842]
Mean Cross-Validation Accuracy: 0.5087115663924767
